In [ ]:
import json
import os
import pandas as pd

In [ ]:
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
DATA_DIR = os.path.join("data", "argentina")
JSON_RESULTS = os.path.join(DATA_DIR, "simulation_results.json")

# Generate data

In [ ]:
import generate_databases
generate_databases.store_all()

In [ ]:
if not os.path.exists('simulation/simulation'):
    !cd simulation && cmake -DCMAKE_BUILD_TYPE=Release . &&make -j8

In [ ]:
!simulation/simulation \
    --days 300 \
    --population {os.path.join(DATA_DIR, "fake_population_small")} \
    --json {JSON_RESULTS} \
    --seed 0

# Show results

In [ ]:
with open(JSON_RESULTS) as f:
  sim_res = json.load(f)
sim_res = pd.DataFrame(sim_res)
sim_res.head()

In [ ]:
states = ["DEAD", "EXPOSED", "INFECTED_1", "INFECTED_2", "INFECTED_3", "RECOVERED", "SUSCEPTIBLE"]

In [ ]:
fig = go.Figure()
for st in states:
    fig.add_trace(go.Scatter(x=sim_res["day"], y=sim_res[st], name=st))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Asdk',
                   xaxis_title='Día',
                   yaxis_title='Número de personas')


fig.show()

In [ ]:
inf_sources = [
    "HOME_CONTACT",
    "SCHOOL_CONTACT",
    "WORK_CONTACT",
    "NEIGHBOURHOOD_CONTACT",
    "INTER_NEIGHBOURHOOD_CONTACT",
    "IMPORTED_CASE"
]

In [ ]:
fig = go.Figure()
for st in inf_sources:
    fig.add_trace(go.Scatter(x=sim_res["day"], y=sim_res[st], name=st))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Fuente de infecciones',
                   xaxis_title='Día',
                   yaxis_title='Número de personas')


fig.show()